In [ ]:
# Required Imports
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import datetime
import warnings
warnings.filterwarnings("ignore")

# Step 1: Define Date Range
end = datetime.datetime.now()
start = end - datetime.timedelta(days=365)

# Step 2: Download Stock Data
df = yf.download('AAPL', start=start, end=end)
df_close = df['Close'].dropna()

# Step 3: Fit ARIMA Model
model = ARIMA(df_close, order=(5, 1, 0))  # (p, d, q)
model_fit = model.fit()

# Step 4: Forecast Future Prices
forecast = model_fit.forecast(steps=10)
future_dates = pd.bdate_range(start=df_close.index[-1] + pd.Timedelta(days=1), periods=10)
forecast_series = pd.Series(forecast.values, index=future_dates)

# Step 5: Plot Observed and Forecasted Prices
plt.figure(figsize=(10, 4))
plt.plot(df_close.index, df_close, label='Observed')
plt.plot(forecast_series.index, forecast_series, label='Forecast', color='red')
plt.title("AAPL Stock Forecast (ARIMA)")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Step 6: Generate Buy/Sell Signals
buy_signals = []
for i in range(1, len(forecast_series)):
    if forecast_series[i] > forecast_series[i - 1]:
        buy_signals.append("Buy")
    else:
        buy_signals.append("Sell")

# Step 7: Create DataFrame of Trading Signals
trading_signals = pd.DataFrame({
    "Date": forecast_series.index[1:],
    "Forecasted Price": forecast_series.values[1:],
    "Signal": buy_signals
})

# Print Trading Signals
print(trading_signals)

# Step 8: Plot Forecasted Prices with Buy/Sell Signals
plt.figure(figsize=(12, 6))
plt.plot(forecast_series, label='Forecasted Price', color='blue', marker='o')

# Annotate Buy/Sell Points
for i, signal in enumerate(buy_signals):
    color = 'green' if signal == 'Buy' else 'red'
    plt.scatter(
        trading_signals['Date'].iloc[i],
        trading_signals['Forecasted Price'].iloc[i],
        color=color,
        label=signal if i == 0 or signal != buy_signals[i - 1] else "",
        s=100,
        edgecolors='black'
    )

plt.title("ARIMA Forecast with Buy/Sell Signals")
plt.xlabel("Date")
plt.ylabel("Forecasted Price")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
